In [1]:
# env : tmp
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from gensim import models
from gensim.models import Word2Vec, FastText
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from kobart_transformers import get_kobart_model, get_kobart_tokenizer
%matplotlib inline

In [ ]:
# summarized data
## lexrank
lexrank = pd.read_csv("data/textrank_lexrank.csv")
lexrank = lexrank[['리뷰자', '본문', 'lexrank1']].copy()
lexrank.columns = ['리뷰자', '본문', 'lexrank']

## kobart
kobart = pd.read_csv("data/kobart_summerize.csv")
kobart.columns = ['리뷰자', '본문', 'kobart']

## chatGPT
gpt = pd.read_csv('data/chatgpt_summerize.csv')
gpt.drop(['Unnamed: 0'], axis=1, inplace=True)
gpt.columns = ['리뷰자', 'gpt']

In [24]:
# gpt 데이터에 본문 컬럼 추가
gpt = gpt.merge(kobart[['리뷰자', '본문']], how = 'inner', on = '리뷰자')
gpt = gpt[['리뷰자', '본문', 'gpt']]
gpt.shape

(3574, 3)

In [25]:
with open("data/product.pickle", 'rb') as p:
    product = pickle.load(p)
with open("data/purchase.pickle", 'rb') as p:
    purchase = pickle.load(p)

# 구매 데이터 중 상품 정보가 있는 것만 저장
df = purchase.merge(product, how = 'inner', right_on='상품명', left_on='상품')
#리뷰자별 리뷰 전문
review = df.groupby('리뷰자')['본문'].sum().reset_index()
review.shape

(3576, 2)

In [26]:
print(lexrank.columns)
print(kobart.columns)
print(gpt.columns)

Index(['리뷰자', '본문', 'lexrank'], dtype='object')
Index(['리뷰자', '본문', 'kobart'], dtype='object')
Index(['리뷰자', '본문', 'gpt'], dtype='object')


In [28]:
# 리뷰가 없는 사용자
insert = review[review['본문'] == ''].reset_index(drop=True)

# 리뷰가 없는 사용자 데이터프레임에 추가
insert_lex = insert.copy()
insert_lex['lexrank'] = ''

insert_kob = insert.copy()
insert_kob['kobart'] = ''

insert_gpt = insert.copy()
insert_gpt['gpt'] = ''

lexrank = pd.concat([lexrank, insert_lex]).reset_index(drop=True)
kobart = pd.concat([kobart, insert_kob]).reset_index(drop=True)
gpt = pd.concat([gpt, insert_gpt]).reset_index(drop=True)
print(lexrank.shape, kobart.shape, gpt.shape)

(3576, 3) (3576, 3) (3576, 3)


In [ ]:
gpt[gpt['gpt'].isna() == True] #gpt로 요약이 안 된 건 확인 : 영어

In [36]:
# 공백
gpt['gpt'] = gpt['gpt'].fillna('')
gpt.isna().sum()

리뷰자    0
본문     0
gpt    0
dtype: int64

### preprocessing
- feature 수를 줄이기 위함

In [29]:
# 토큰화
def okt_tokenize(txt):
    okt = Okt()
    k = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', "", txt) # 한글 및 띄어쓰기만 남기기
    words = okt.pos(k) #토큰화
    words = [w[0] for w in words if len(w[0]) > 2] #3글자 이상의 토큰만 남기기
    return words

In [64]:
# 토큰화 함수 구현 : lexrank
lexrank_list = list()

for i,j in enumerate(lexrank['lexrank']):
    lexrank_list.append(okt_tokenize(j))
    if i % 100 == True:
        print(i, "완료")

1 완료
101 완료
201 완료
301 완료
401 완료
501 완료
601 완료
701 완료
801 완료
901 완료
1001 완료
1101 완료
1201 완료
1301 완료
1401 완료
1501 완료
1601 완료
1701 완료
1801 완료
1901 완료
2001 완료
2101 완료
2201 완료
2301 완료
2401 완료
2501 완료
2601 완료
2701 완료
2801 완료
2901 완료
3001 완료
3101 완료
3201 완료
3301 완료
3401 완료
3501 완료


In [65]:
lex_token_temp = sum(lexrank_list, [])
lex_count = dict(Counter(lex_token_temp))
len(lex_count) #총 8420개 토큰(2글자 토큰 제외로 인한 토큰 개수 감소) : 12743 > 8420

8420

In [66]:
lex_token = pd.DataFrame(list(lex_count.items()), columns=['token', 'count'])
lex_token[lex_token['count'] == 1].shape # count가 1인 경우 : 과반

(5278, 2)

In [68]:
# 토큰 count가 2 이상인 경우의 토큰만 저장
lex_token = list(lex_token[lex_token['count'] != 1]['token'])
lex_token

['따가웠는데',
 '좋아용',
 '드렸는데',
 '괜찮은',
 '샀어요',
 '했더니',
 '나간다고',
 '하셨어요',
 '좋다고',
 '하시네요',
 '괜찮네요',
 '트러블',
 '하려고',
 '여드름',
 '밸런스',
 '줄었어요',
 '만족스럽습니다',
 '사이즈',
 '딱입니다',
 '에멀전',
 '바르는게',
 '불편해요',
 '좋은데',
 '꾸준히',
 '올인원',
 '이에요',
 '좋습니다',
 '쓰더라구요',
 '피부관리',
 '입니다',
 '순하고',
 '여름철',
 '쓰겠어요',
 '순하게',
 '같아요',
 '있었는데',
 '있어서',
 '바르면',
 '뒤집어지거나',
 '없었어서',
 '쓰셔도',
 '적합할',
 '많아서',
 '좋아요',
 '해주며',
 '손바닥',
 '적당량',
 '바릅니다',
 '해보니까',
 '하고싶은',
 '해보고',
 '하고나서',
 '재구매',
 '사줬는데',
 '어머니',
 '화장품',
 '떨어져서',
 '했어요',
 '약해서',
 '내용물',
 '왔네요',
 '만족합니다',
 '떨어져가서',
 '합니다',
 '부드럽게',
 '발려서',
 '순해요',
 '마스크',
 '건조해져서',
 '했는데',
 '좋아서',
 '있어요',
 '이라서',
 '같아서',
 '바르고',
 '부족해서',
 '다가올',
 '가볍고',
 '하는지',
 '몰라요',
 '촉촉하고',
 '많습니다',
 '당기는',
 '봤는데',
 '쓰지만',
 '저렴해서',
 '바르기',
 '신제품',
 '이라고',
 '알았는데',
 '올리브영',
 '좋네요',
 '있는줄',
 '몰랐는데',
 '잘쓸게요',
 '필요했는데',
 '붙어있어서',
 '좋았어요',
 '갑자기',
 '속상했는데',
 '클렌저',
 '해줬더니',
 '좋아졌어요',
 '없어서',
 '촉촉해요',
 '좋을것',
 '유튜버',
 '브랜드',
 '사보았어요',
 '순하기',
 '좋아지는',
 '아니고',
 '없었어요',
 '잘맞는',
 '하는데',
 '아쉽네요',
 '이시고',
 '

In [69]:
# 토큰화 함수 구현 : kobart
kobart_list = list()

for i,j in enumerate(kobart['kobart']):
    kobart_list.append(okt_tokenize(j))
    if i % 100 == True:
        print(i, "완료")

1 완료
101 완료
201 완료
301 완료
401 완료
501 완료
601 완료
701 완료
801 완료
901 완료
1001 완료
1101 완료
1201 완료
1301 완료
1401 완료
1501 완료
1601 완료
1701 완료
1801 완료
1901 완료
2001 완료
2101 완료
2201 완료
2301 완료
2401 완료
2501 완료
2601 완료
2701 완료
2801 완료
2901 완료
3001 완료
3101 완료
3201 완료
3301 완료
3401 완료
3501 완료


In [70]:
kob_token_temp = sum(kobart_list, [])
kob_count = dict(Counter(kob_token_temp))
len(kob_count) #총 6912개 토큰

6912

In [71]:
kob_token = pd.DataFrame(list(kob_count.items()), columns=['token', 'count'])
kob_token[kob_token['count'] == 1].shape # count가 1인 경우

(3120, 2)

In [72]:
# 토큰 count가 2 이상인 경우의 토큰만 저장
kob_token = list(kob_token[kob_token['count'] != 1]['token'])
kob_token

['따가웠는데',
 '일어나도',
 '보송하게',
 '되어서',
 '좋고대',
 '드렸는데',
 '만족한다며',
 '좋아하시네요',
 '괜찮은',
 '샀어요',
 '했더니',
 '나간다고',
 '하셨어요',
 '드렸어요',
 '트러블',
 '러러러',
 '사이즈',
 '에멀전',
 '바르는게',
 '불편해요',
 '좋은데',
 '썼는데',
 '했어요',
 '꾸준히',
 '쓰더라구요',
 '피부관리',
 '입니다',
 '순하고',
 '좋습니다',
 '여름철',
 '쓰겠어요',
 '많아서',
 '하기에도',
 '괜찮고',
 '되지만',
 '않아서',
 '아니면',
 '하더라고요',
 '여드름',
 '있었는데',
 '같아요',
 '있어서',
 '바르면',
 '손바닥',
 '적당량',
 '덜어내어',
 '바릅니다',
 '싫어해서',
 '테스트',
 '해보고',
 '사준건데',
 '어머니',
 '화장품',
 '떨어져서',
 '쓰시던',
 '됐는지',
 '됐는데',
 '약해서',
 '내용물',
 '만족합니다',
 '건조해져서',
 '했는데',
 '좋아서',
 '있어요',
 '썼었는데',
 '순해요',
 '다가오면서',
 '같아서',
 '민감하고',
 '예민한',
 '보세요',
 '다가올',
 '가볍고',
 '여러분',
 '많습니다',
 '쓰지만',
 '좋아요',
 '촉촉해요',
 '없어서',
 '좋어용',
 '바르기',
 '저렴해서',
 '패키지',
 '바뀌었을까',
 '바뀌었을까요',
 '신제품',
 '이라고',
 '나온줄',
 '알았는데',
 '알았다가',
 '알았음',
 '알았단',
 '알았다고',
 '나온줄음',
 '봤지만',
 '새로운거라',
 '사봄에',
 '사봄라',
 '라라라',
 '올리브영',
 '좋어요',
 '유튜버',
 '브랜드',
 '갑자기',
 '쓰는데',
 '맞는것',
 '입니당',
 '적셔서',
 '맞을듯',
 '묻혀서',
 '해주기도',
 '먹어요',
 '다양하게',
 '촉촉하고',
 '무겁고',
 '싫어하는데',
 '은은해서',
 '바르고'

In [37]:
# 토큰화 함수 구현 : gpt
gpt_list = list()

for i,j in enumerate(gpt['gpt']):
    gpt_list.append(okt_tokenize(j))
    if i % 100 == True:
        print(i, "완료")

1 완료
101 완료
201 완료
301 완료
401 완료
501 완료
601 완료
701 완료
801 완료
901 완료
1001 완료
1101 완료
1201 완료
1301 완료
1401 완료
1501 완료
1601 완료
1701 완료
1801 완료
1901 완료
2001 완료
2101 완료
2201 완료
2301 완료
2401 완료
2501 완료
2601 완료
2701 완료
2801 완료
2901 완료
3001 완료
3101 완료
3201 완료
3301 완료
3401 완료
3501 완료


In [38]:
gpt_token_temp = sum(gpt_list, [])
gpt_count = dict(Counter(gpt_token_temp))
len(gpt_count) #총 10563개 토큰

10563

In [39]:
gpt_token = pd.DataFrame(list(gpt_count.items()), columns=['token', 'count'])
gpt_token[gpt_token['count'] == 1].shape # count가 1인 경우 : 과반

(6418, 2)

In [40]:
# 토큰 count가 2 이상인 경우의 토큰만 저장
gpt_token = list(gpt_token[gpt_token['count'] != 1]['token'])
gpt_token

['따가웠는데',
 '일어나도',
 '보송하게',
 '되어서',
 '좋아용',
 '드렸는데',
 '좋아하시네요',
 '괜찮은',
 '샀어요',
 '했더니',
 '나간다고',
 '하셨어요',
 '좋다고',
 '하시네요',
 '괜찮네요',
 '트러블',
 '하려고',
 '올인원',
 '이라서',
 '편할거',
 '같네요',
 '여드름',
 '썼는데',
 '했어요',
 '밸런스',
 '줄었어요',
 '만족스럽습니다',
 '사이즈',
 '딱입니다',
 '에멀전',
 '바르는게',
 '불편해요',
 '좋은데',
 '꾸준히',
 '이에요',
 '넘치는',
 '좋습니다',
 '쓰더라구요',
 '피부관리',
 '입니다',
 '순하고',
 '여름철',
 '쓰겠어요',
 '순하게',
 '같아요',
 '많아서',
 '하기에도',
 '괜찮고',
 '됩니다',
 '않아서',
 '아니면',
 '하더라고요',
 '있었는데',
 '있어서',
 '바르면',
 '뒤집어지거나',
 '없었어서',
 '쓰셔도',
 '적합할',
 '좋아요',
 '해주며',
 '손바닥',
 '적당량',
 '바릅니다',
 '거칠어진',
 '탄탄하고',
 '건강한',
 '그리고',
 '시키고',
 '싶었는데',
 '이렇게',
 '생겼네요',
 '해봤는데',
 '충분히',
 '올려놓고',
 '무겁지',
 '가벼워서',
 '좋네요',
 '해보니까',
 '하고싶은',
 '해보고',
 '하고나서',
 '재구매',
 '화장품',
 '부작용',
 '전문의',
 '사줬는데',
 '싫어해서',
 '테스트',
 '사준건데',
 '어머니',
 '떨어져서',
 '쓰시던',
 '됐는지',
 '됐는데',
 '약해서',
 '내용물',
 '왔네요',
 '나머진',
 '만족합니다',
 '떨어져가서',
 '합니다',
 '부드럽게',
 '발려서',
 '순해요',
 '마스크',
 '건조해져서',
 '했는데',
 '좋아서',
 '있어요',
 '썼었는데',
 '같아서',
 '바르고',
 '민감하고',
 '예민한',
 '보세요',
 '부족해서',
 '다가올'

### TF-IDF

In [126]:
lexrank['lexrank_clean'] = np.nan
words = [d for d in lexrank_list]

for i in range(len(words)):
    temp = list()
    for j in words[i]:
        if j in lex_token: #lex token에 있는 토큰만 저장
            temp.append(j)
    lexrank.loc[i, 'lexrank_clean'] = ' '.join(temp)

In [130]:
print(lexrank.shape)
lexrank.isna().sum()

(3576, 4)


리뷰자              0
본문               0
lexrank          0
lexrank_clean    0
dtype: int64

In [131]:
kobart['kobart_clean'] = np.nan
words = [d for d in kobart_list]

for i in range(len(words)):
    temp = list()
    for j in words[i]:
        if j in kob_token: #kob token에 있는 토큰만 저장
            temp.append(j)
    kobart.loc[i, 'kobart_clean'] = ' '.join(temp)

In [133]:
print(kobart.shape)
kobart.isna().sum()

(3576, 4)


리뷰자             0
본문              0
kobart          0
kobart_clean    0
dtype: int64

In [41]:
gpt['gpt_clean'] = np.nan
words = [d for d in gpt_list]

for i in range(len(words)):
    temp = list()
    for j in words[i]:
        if j in gpt_token: #gpt token에 있는 토큰만 저장
            temp.append(j)
    gpt.loc[i, 'gpt_clean'] = ' '.join(temp)

In [43]:
gpt.isna().sum()

리뷰자          0
본문           0
gpt          0
gpt_clean    0
dtype: int64

In [44]:
# embedding : tf-idf
def tfidf(txt):
    ### tfidf vectorizer 생성 및 적용
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(txt)
    tfidf = pd.DataFrame(tfidf.toarray(), columns = vectorizer.get_feature_names_out())
    return tfidf

In [ ]:
lexrank_tfidf = tfidf(lexrank['lexrank_clean'])
lexrank_tfidf.index = lexrank['리뷰자']
lexrank_tfidf

In [ ]:
kobart_tfidf = tfidf(kobart['kobart_clean'])
kobart_tfidf.index = kobart['리뷰자']
kobart_tfidf

In [ ]:
gpt_tfidf = tfidf(gpt['gpt_clean'])
gpt_tfidf.index = gpt['리뷰자']
gpt_tfidf

In [46]:
# csv로 저장
lexrank_tfidf.to_csv("data/lexrank_tfidf2.csv", encoding='utf-8-sig', index=True)
kobart_tfidf.to_csv("data/kobart_tfidf.csv", encoding='utf-8-sig', index=True)
gpt_tfidf.to_csv("data/gpt_tfidf.csv", encoding='utf-8-sig', index=True)

### Word2Vec
- reference : https://heung-bae-lee.github.io/2020/01/30/NLP_04/

In [140]:
lexrank_list

[['따가웠는데', '좋아용'],
 ['드렸는데', '괜찮은', '샀어요', '했더니', '주셨고', '나간다고', '하셨어요', '좋다고', '하시네요', '괜찮네요'],
 ['트러블', '하려고', '샀어요'],
 ['여드름',
  '밸런스',
  '맞아지고',
  '트러블',
  '줄었어요',
  '만족스럽습니다',
  '사이즈',
  '딱입니다',
  '에멀전',
  '바르는게',
  '불편해요',
  '좋은데'],
 ['꾸준히',
  '올인원',
  '이에요',
  '좋습니다',
  '꾸준히',
  '쓰더라구요',
  '피부관리',
  '입니다',
  '순하고',
  '좋습니다',
  '여름철',
  '쓰겠어요'],
 ['순하게',
  '같아요',
  '있었는데',
  '같아요',
  '있어서',
  '바르면',
  '같아요',
  '뒤집어지거나',
  '없었어서',
  '여드름',
  '쓰셔도',
  '적합할',
  '갘습니',
  '많아서',
  '좋아요'],
 ['해주며',
  '손바닥',
  '적당량',
  '덜어내어',
  '바릅니다',
  '해보니까',
  '하고싶은',
  '꾸준히',
  '해보고',
  '하고나서',
  '만족스러우면',
  '재구매'],
 ['사줬는데', '좋아졌다며', '만족하더라고요'],
 ['어머니',
  '화장품',
  '떨어져서',
  '했어요',
  '약해서',
  '에어캡',
  '터지고',
  '내용물',
  '왔네요',
  '나머진',
  '만족합니다',
  '떨어져가서',
  '재구매',
  '합니다',
  '순하고',
  '같아요',
  '떨어져가서',
  '재구매',
  '합니다',
  '부드럽게',
  '발려서',
  '순해요'],
 ['마스크', '건조해져서', '했는데', '좋아서', '있어요'],
 ['이라서', '좋아요'],
 ['다가오면서', '같아서', '바르고', '있어요'],
 ['좋아요', '부족해서', '발라주긴', '다가올', '가볍고', '좋다구요', '하는지', '몰라요',

In [199]:
# embedding : word2vec
model1 = Word2Vec(sentences=lexrank_list, vector_size = 100, window = 5, workers = 6, sg = 0, seed = 42)
print(model1.wv.vectors.shape)
model1.wv.vectors

(1184, 100)


array([[ 2.62509048e-01, -1.10292330e-01, -1.76539347e-01, ...,
         2.91046977e-01, -3.79152864e-01, -1.06657460e-01],
       [ 3.14919859e-01, -1.39821723e-01, -1.99680224e-01, ...,
         3.19912910e-01, -4.38818872e-01, -1.03086926e-01],
       [ 2.83594936e-01, -1.14557952e-01, -1.90573156e-01, ...,
         3.01778287e-01, -4.07153040e-01, -1.13464065e-01],
       ...,
       [ 2.84186061e-02, -1.86252501e-02, -3.01295053e-02, ...,
         3.72245088e-02, -5.75959608e-02, -1.09505458e-02],
       [ 4.48525399e-02, -2.21183840e-02, -3.04637663e-02, ...,
         4.23224941e-02, -5.98744154e-02, -2.23801360e-02],
       [ 1.40326833e-02,  9.94865841e-05, -8.99888389e-03, ...,
         2.34229248e-02, -2.60216463e-02, -3.54506029e-03]], dtype=float32)

In [50]:
def get_features(words, model, num_features):
  # 출력 벡터 초기화
  feature_vector = np.zeros((num_features), dtype=np.float32)
  num_words = 0
  # 어휘사전 준비
  index2word_set = set(model.wv.index_to_key)
  for w in words:
    if w in index2word_set:
      num_words +=1
      # 사전에 해당하는 단어에 대해 단어 벡터를 더함
      feature_vector = np.add(feature_vector, model.wv[w])
  # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
  feature_vector = np.divide(feature_vector, num_words)
  return feature_vector

def get_dataset(reviews, model, num_features):
  dataset = list()
  for s in reviews:
    dataset.append(get_features(s, model, num_features))
  reviewFeatureVecs = np.stack(dataset)
  return reviewFeatureVecs

In [ ]:
lexrank_word2vec = pd.DataFrame(get_dataset(lexrank_list, model1, 100)).fillna(0)
lexrank_word2vec.index = lexrank['리뷰자']
lexrank_word2vec

In [215]:
kobart_list

[['따가웠는데',
  '일어나도',
  '보송하게',
  '되어서',
  '좋고대',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '일어나도',
  '보송하게',
  '되어서',
  '좋고대',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '일어나도',
  '보송하게',
  '되어서',
  '좋고대',
  '따가웠는데',
  '따가웠는데',
  '따가웠는데',
  '일어나도',
  '보송하게',
  '되어서',
  '좋고대'],
 ['드렸는데',
  '만족한다며',
  '좋아하시네요',
  '괜찮은',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '만족한다며',
  '좋아하시네요',
  '괜찮은',
  '샀어요',
  '했더니',
  '주셨고',
  '나간다고',
  '하셨어요',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸어요',
  '드렸어요',
  '드렸어요',
  '드렸어요',
  '드렸어요',
  '드렸어요',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸는데',
  '드렸어요',
  '드렸어요',
  '드렸어요',
  '드렸는데',
  '드렸는데',
  '드렸는데',


In [216]:
model2 = Word2Vec(sentences=kobart_list, vector_size = 100, window = 5, workers = 6, sg = 0, seed = 42)
print(model2.wv.vectors.shape)
model2.wv.vectors

(2154, 100)


array([[ 0.45108852, -0.2454345 , -0.53346324, ...,  0.82427734,
         0.9969071 , -0.98610234],
       [ 1.5042825 , -2.1165767 , -0.65162116, ...,  0.86368686,
         0.7445125 ,  0.57079345],
       [ 0.57201904, -0.97316754, -1.0145822 , ..., -0.7257467 ,
         0.22621812, -0.04542574],
       ...,
       [ 0.1280622 , -0.06940389, -0.02830932, ...,  0.03354751,
         0.12304012,  0.02795968],
       [ 0.04939971, -0.03766453, -0.0229879 , ...,  0.0193411 ,
         0.06788797, -0.0100893 ],
       [ 0.24639936, -0.17417598, -0.05982855, ...,  0.07242544,
         0.2487693 ,  0.04417985]], dtype=float32)

In [ ]:
kobart_word2vec = pd.DataFrame(get_dataset(kobart_list, model2, 100)).fillna(0)
kobart_word2vec.index = kobart['리뷰자']
kobart_word2vec

In [47]:
gpt_list

[['따가웠는데', '일어나도', '보송하게', '되어서', '좋아용'],
 ['드렸는데',
  '만족한다며',
  '좋아하시네요',
  '괜찮은',
  '샀어요',
  '했더니',
  '주셨고',
  '나간다고',
  '하셨어요',
  '좋다고',
  '하시네요',
  '괜찮네요'],
 ['트러블', '하려고', '샀어요', '올인원', '이라서', '편할거', '같네요'],
 ['여드름',
  '썼는데',
  '했어요',
  '밸런스',
  '맞아지고',
  '트러블',
  '줄었어요',
  '만족스럽습니다',
  '사이즈',
  '딱입니다',
  '에멀전',
  '바르는게',
  '불편해요',
  '좋은데'],
 ['꾸준히',
  '올인원',
  '이에요',
  '넘치는',
  '이에요',
  '좋습니다',
  '꾸준히',
  '쓰더라구요',
  '피부관리',
  '입니다',
  '순하고',
  '좋습니다',
  '여름철',
  '쓰겠어요'],
 ['순하게',
  '같아요',
  '많아서',
  '하기에도',
  '괜찮고',
  '됩니다',
  '않아서',
  '쟁여야',
  '아니면',
  '하더라고요',
  '여드름',
  '있었는데',
  '같아요',
  '있어서',
  '바르면',
  '같아요',
  '뒤집어지거나',
  '없었어서',
  '여드름',
  '쓰셔도',
  '적합할',
  '갘습니',
  '많아서',
  '좋아요'],
 ['해주며',
  '손바닥',
  '적당량',
  '덜어내어',
  '바릅니다',
  '쇼듐헤파린',
  '거칠어진',
  '탄탄하고',
  '건강한',
  '궁금했었고',
  '브루스',
  '그리고',
  '있어서',
  '시키고',
  '싶었는데',
  '이렇게',
  '생겼네요',
  '해봤는데',
  '충분히',
  '올려놓고',
  '시켜놨어요',
  '무겁지',
  '가벼워서',
  '좋네요',
  '해보니까',
  '하고싶은',
  '꾸준히',
  '해보고',
  '하고나서',
  '만족스러우면',
  

In [48]:
model3 = Word2Vec(sentences=gpt_list, vector_size = 100, window = 5, workers = 6, sg = 0, seed = 42)
print(model3.wv.vectors.shape)
model3.wv.vectors

(1616, 100)


array([[ 0.2823361 , -0.09759446, -0.27649674, ...,  0.21708016,
        -0.42909178, -0.07025172],
       [ 0.34031558, -0.12083986, -0.30976313, ...,  0.24077098,
        -0.49738246, -0.06153715],
       [ 0.3199977 , -0.10348078, -0.3054086 , ...,  0.23497967,
        -0.473876  , -0.07768904],
       ...,
       [ 0.03191755, -0.01763963, -0.04537923, ...,  0.01996979,
        -0.04838482, -0.00769515],
       [ 0.03903082, -0.01674369, -0.0190093 , ...,  0.02505465,
        -0.03751387, -0.0071004 ],
       [ 0.06049421, -0.02703648, -0.05524655, ...,  0.03238249,
        -0.0894222 , -0.0059588 ]], dtype=float32)

In [ ]:
gpt_word2vec = pd.DataFrame(get_dataset(gpt_list, model3, 100)).fillna(0)
gpt_word2vec.index = gpt['리뷰자']
gpt_word2vec

In [52]:
# csv로 저장
lexrank_word2vec.to_csv("data/lexrank_word2vec.csv", encoding='utf-8-sig', index=True)
kobart_word2vec.to_csv("data/kobart_word2vec.csv", encoding='utf-8-sig', index=True)
gpt_word2vec.to_csv("data/gpt_word2vec.csv", encoding='utf-8-sig', index=True)

### kobart

In [53]:
kobart_tokenizer = get_kobart_tokenizer()
model = get_kobart_model()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [54]:
def get_embedding(txt, num = 768):
    embeddings = []
    tokens = kobart_tokenizer.tokenize(txt)
    input_ids = kobart_tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor([input_ids])
    try:
        with torch.no_grad():
            lhs = model(input_ids)[0].mean(dim=1) #평균 이용
    except IndexError: #리뷰가 없는 경우 0으로 채워진 array 생성
        lhs = np.zeros(num)
    embeddings.append(lhs.tolist())
    return embeddings[0]

In [114]:
columns = [f"embedding_{i}" for i in range(768)]
df = pd.DataFrame(columns=columns)

for i,j in enumerate(lexrank['lexrank']):
    df.loc[i] = get_embedding(j)[0]
    if i % 100 == True:
        print(i, "완료")

1 완료
101 완료
201 완료
301 완료
401 완료
501 완료
601 완료
701 완료
801 완료
901 완료
1001 완료
1101 완료
1201 완료
1301 완료
1401 완료
1501 완료
1601 완료
1701 완료
1801 완료
1901 완료
2001 완료
2101 완료
2201 완료
2301 완료
2401 완료
2501 완료
2601 완료
2701 완료
2801 완료
2901 완료
3001 완료
3101 완료
3201 완료
3301 완료
3401 완료
3501 완료


In [ ]:
df.index = lexrank['리뷰자']

In [116]:
# csv로 저장
df.to_csv("data/lexrank_emb_kobart.csv", encoding='utf-8-sig', index=True)

In [121]:
columns = [f"embedding_{i}" for i in range(768)]
df2 = pd.DataFrame(columns=columns)

for i,j in enumerate(kobart['kobart']):
    df2.loc[i] = get_embedding(j)[0]
    if i % 100 == True:
        print(i, "완료")

1 완료
101 완료
201 완료
301 완료
401 완료
501 완료
601 완료
701 완료
801 완료
901 완료
1001 완료
1101 완료
1201 완료
1301 완료
1401 완료
1501 완료
1601 완료
1701 완료
1801 완료
1901 완료
2001 완료
2101 완료
2201 완료
2301 완료
2401 완료
2501 완료
2601 완료
2701 완료
2801 완료
2901 완료
3001 완료
3101 완료
3201 완료
3301 완료
3401 완료
3501 완료


In [ ]:
df2.index = kobart['리뷰자']

In [123]:
# csv로 저장
df2.to_csv("data/kobart_emb_kobart.csv", encoding='utf-8-sig', index=True)

In [55]:
columns = [f"embedding_{i}" for i in range(768)]
df3 = pd.DataFrame(columns=columns)

for i,j in enumerate(gpt['gpt']):
    df3.loc[i] = get_embedding(j)[0]
    if i % 100 == True:
        print(i, "완료")

1 완료
101 완료
201 완료
301 완료
401 완료
501 완료
601 완료
701 완료
801 완료
901 완료
1001 완료
1101 완료
1201 완료
1301 완료
1401 완료
1501 완료
1601 완료
1701 완료
1801 완료
1901 완료
2001 완료
2101 완료
2201 완료
2301 완료
2401 완료
2501 완료
2601 완료
2701 완료
2801 완료
2901 완료
3001 완료
3101 완료
3201 완료
3301 완료
3401 완료
3501 완료


In [ ]:
df3.index = gpt['리뷰자']

In [57]:
# csv로 저장
df3.to_csv("data/gpt_emb_kobart.csv", encoding='utf-8-sig', index=True)